<h1 align=center><font size = 5>Segmenting and Clustering Neighborhouds in Toronto</font></h1>
<h1 align=center><font size = 2>Ilan Benchetrit</font></h1>

# I - Scrapping data

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: - 
  - anaconda/osx-64::ca-certificates-2020.1.1-0, anaconda/osx-64::openssl-1.1.1d-h1de35cc_4
  - anaconda/osx-64::ca-certificates-2020.1.1-0, defaults/osx-64::openssl-1.1.1d-h1de35cc_4
  - anaconda/osx-64::openssl-1.1.1d-h1de35cc_4, defaults/osx-64::ca-certificates-2020.1.1-0
  - defaults/osx-64::ca-certificates-2020.1.1-0, defaults/osx-64::openssl-1.1.1d-h1de35ccdone

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py37_0         148 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         148 KB

The following packages will be UPDATED:

  conda                        anaconda::conda-4.8.3-py37_0 --> conda-forge::conda-4.8.3-py37hc8dfbb8_1

The fol

Now let's import Beautiful Soup and its dependecies to scrape the Wikipedia page

In [2]:
!conda install -c anaconda beautifulsoup4 --yes
from bs4 import BeautifulSoup

!conda install -c anaconda lxml --yes
!conda install -c anaconda html5lib --yes
!conda install -c anaconda requests --yes
import requests

print('BeautifulSoup and its dependecies imported')

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py37_0         156 KB  anaconda
    ------------------------------------------------------------
                                           Total:         156 KB

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi                                       conda-forge --> anaconda
  conda              conda-forge::conda-4.8.3-py37hc8dfbb8~ --> anaconda::conda-4.8.3-py37_0



certifi-2019.11.28   | 156 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages already installed.

Solving environment: done

# All req

Now let's import Geocoder and its dependecies to easily retrieve locations

In [3]:
%pip install geocoder
print('Geocoder installed')

Note: you may need to restart the kernel to use updated packages.
Geocoder installed


#### Load the html page and scrap it with BeautifulSoup

In [131]:
!wget -q -O 'toronto_data.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Data downloaded!')

Data downloaded!


In [132]:
with open("toronto_data.html") as html_file:
    wikipage = BeautifulSoup(html_file,'lxml')

body = wikipage.find('tbody')

#print(body.prettify())

Then, we extract the usefull data within the HTML page.
<br>In the following code, we assumed that : 
- if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
- if a cell has a neighbourhood but a Not assigned borough, then the borough will be the same as the neighbourhood (M7A for instance). **EDIT : this code was written before knowing M7A is populated in the location file.**
- as it is not requested, we get rid of cardinal specifications

In [139]:
data = []
for cell in body.find_all('p'):
    #we first extract the postal code
    pcode = cell.b.text
    
    #then we extract the borough and the neighbourhood
    try :
        try :
            borough = cell.i.text #in this case, the borough is not assigned so it is formated in italic
            neighbourhood = borough
        except :
            borough = cell.span.text.split('(')[0] 
            neighbourhood = cell.span.text.split('(')[1] #we split borough from neighbourhoods
            neighbourhood = neighbourhood.split(')')[0] #we get rid of cardinal specifications
            neighbourhood = neighbourhood.replace(' /',',')
    except : #this case is for postal code without borough like M7A
        borough = 'Not assigned'
        neighbourhood = cell.span.text
        neighbourhood = neighbourhood.split(')')[0] #we get rid of cardinal specifications
        neighbourhood = neighbourhood.replace(' /',',')
    
    #we append this instance of the loop into the postal_code list
    l = [pcode, borough, neighbourhood]
    data.append(l)

#print(data)

#### Tranform the data into a *pandas* dataframe

In [140]:
# define the dataframe columns
column_names = ['Postal Code', 'Borough', 'Neighbourhood'] 

# instantiate the dataframe
df = pd.DataFrame(columns=column_names)

#df

In [141]:
for l in data:
    postal_code = l[0]
    borough = l[1]
    neighbourhood = l[2]
    
    df = df.append({'Postal Code': postal_code,
                    'Borough': borough,
                    'Neighbourhood': neighbourhood}, 
                   ignore_index=True)

#df

**Here is the adding of a Borough for M7A Postal Code**

In [142]:
df.loc[df['Postal Code'] == 'M7A','Borough'] = 'Downtown Toronto'

Now we deletre rows for which Borough is not assigned


In [143]:
indexNames = df[ df['Borough'] == 'Not assigned' ].index
df.drop(indexNames , inplace=True)

As requested, here is the shape of the final dataframe with clean data

In [144]:
df.shape

(103, 3)

# II - Scrapping locations

We are using the CSV file to retreive the coordinates of each postal code

In [33]:
import geocoder # import geocoder

#for i in range(len(df)):
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(df.iloc[0]['Postal Code']))
    lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

df = df.append({'Latitude': latitude,
                'Longitude': longitude}, ignore_index=True)

df

KeyboardInterrupt: 

First, we download the location data

In [39]:
!wget -q -O 'toronto_localisation.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [55]:
loc_data = pd.read_csv("toronto_localisation.csv")
loc_data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [70]:
clean_df = pd.merge(left=df, right=loc_data, how='right', left_on='Postal Code', right_on='Postal Code')
clean_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
5,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
6,M3B,North York,Don Mills,43.745906,-79.352188
7,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
8,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
9,M6B,North York,Glencairn,43.709577,-79.445073


In [53]:
clean_df.shape

(102, 5)